We applied the 3DSC methodology https://github.com/aimat-lab/3DSC to handle chemical formulas, including the definitions for exact matching, similarities, doping, and unmatched cases.

In [1]:
from pymatgen.core import Structure
from pymatgen.io.cif import CifWriter
from pymatgen.transformations.standard_transformations import OrderDisorderedStructureTransformation
order = OrderDisorderedStructureTransformation()
import numpy as np
from structure import get_chem_dict,get_formula_similarity,check_for_doping,get_doping_structure
def test_similarity(formula_sc, formula_2):
    chemdict_sc = get_chem_dict(formula_sc)
    chemdict_2 = get_chem_dict(formula_2)
    return  get_formula_similarity(chemdict_sc, chemdict_2)
test_similarity('Ba3Ca1Cu6.58Fe0.42La2.5Y0.5O16.599','Ba1.286Ca0.429Cu2.764Fe0.236La1.071Y0.214O7.24')

(2, 0.016915842183891423)

In [2]:
import os
fn='cif/'
names={}
for name in os.listdir(fn):
    structure=Structure.from_file(fn+name)
    reference_formula=structure.formula.replace(" ","")
    names[name]=reference_formula #cifs names

Match cifs with Chemical formulas in Supercon.

The order transformation method retains only ordered structures with the lowest Ewald energy, to generate ordered cifs.

In [3]:
from pymatgen.core import Structure
from pymatgen.transformations.standard_transformations import OrderDisorderedStructureTransformation
order = OrderDisorderedStructureTransformation()
import openpyxl
workbook = openpyxl.load_workbook('SuperBand.xlsx')	
sheet = workbook['Sheet1']  
for ii in range(10):
    print("ii=",ii)
    for i in range (2,len(sheet["A"])+1):
        if not sheet["B"+str(i)].value:
            continue
        tmp=True
        #print(sheet["B"+str(i)].value)
        for name in os.listdir(fn):
            try:
                index,_=test_similarity(sheet["B"+str(i)].value,names[name])
                #index=1:exact matching, 2:similarity, 3:doping, 4:unmatch
            except:
                continue
            if sheet["I"+str(i)].value =='found':
                tmp=False
                continue
            if index<2.1:
                sheet["H"+str(i)]=os.path.splitext(name)[0]
                tmp=False
                print(sheet["B"+str(i)].value,name,'found')
                sheet["I"+str(i)]='found'
                break
        if tmp:
            name=str(sheet["G"+str(i)].value)+'.cif'
            try:
                structure=Structure.from_file(fn+name)
            except:
                print(sheet["B"+str(i)].value,str(sheet["G"+str(i)].value),'nofound')
                sheet["I"+str(i)]='nofound'
                continue
            if check_for_doping(structure): 
                if len(structure)>40:#too many atoms to be doped
                    print(sheet["B"+str(i)].value,name,'huge')
                    sheet["H"+str(i)]=os.path.splitext(name)[0]
                    sheet["I"+str(i)]='huge'
                    continue
                try:
                    structure=order.apply_transformation(structure,return_ranked_list=100)[0]["structure"]
                except:
                    sheet["H"+str(i)]=os.path.splitext(name)[0]
                    print(sheet["B"+str(i)].value,str(sheet["A"+str(i)].value),'error')
                    sheet["I"+str(i)]='error'
                    continue
            cif=get_doping_structure(sheet["B"+str(i)].value,structure)
            try:
                index,_=test_similarity(sheet["B"+str(i)].value,cif.formula.replace(" ",""))
            except:
                index=4
            if index<2.1:
                sheet["H"+str(i)]=sheet["A"+str(i)].value
                print(sheet["B"+str(i)].value,str(sheet["A"+str(i)].value),'doped')
                sheet["I"+str(i)]='doped'
            else:
                sheet["H"+str(i)]=sheet["A"+str(i)].value
                print(sheet["B"+str(i)].value,str(sheet["A"+str(i)].value),'next')
                sheet["I"+str(i)]='next'
            output_cif = CifWriter(cif)
            output_cif.write_file(fn+str(sheet["A"+str(i)].value)+'.cif')
            names[str(sheet["A"+str(i)].value)+'.cif']=cif.formula.replace(" ","")

ii= 0
Ag0.002Al0.998 10196.cif found
Ag0.005Zn0.995 15771.cif found
Ag0.07Sn0.9Te1 16067 doped
Ag0.153Ba1.99Cu2.847Y1O6.7 16068 next
Ag0.1Sn0.87Te1 16067.cif found
Ag0.2In0.8Te1 16070 next
Ag0.2Sn0.8Te1 10008.cif found
Ag0.5Al0.5Th2 16072.cif found
Ag0.775Sn0.225 10053.cif found
Ag0.795Sn0.205 16074 doped
Ag0.815Sn0.185 16075 doped
Ag0.825Sn0.175 16075.cif found
Ag0.828Ga0.172 16077 doped
Ag0.82Sn0.18 16075.cif found
Ag0.835Sn0.165 16075.cif found
Ag0.845Sb0.155 16080 doped
Ag0.845Sn0.155 16081 next
Ag0.865Sb0.135 16080.cif found
Ag0.875Ga0.125 16083 doped
Ag0.885Sb0.115 16084 doped
Ag0.94Tl0.06 16085 doped
Ag0.9Ga0.1 16083.cif found
Ag2In1 16087 next
Al0.004Cu0.996La1.85Sr0.15O4 13447.cif found
Al0.008Cu0.992La1.85Sr0.15O4 13447.cif found
Al0.012Cu0.988La1.85Sr0.15O4 13447.cif found
Al0.013Ge0.237V0.75 14084.cif found
Al0.015Cu0.985La1.85Sr0.15O4 13447.cif found
Al0.018Cu0.982La1.85Sr0.15O4 13447.cif found
Al0.021Cu0.979La1.85Sr0.15O4 13447.cif found
Al0.024B2Mg0.815 16095 next
Al0.02